In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")

import sqlite3
import pandas as pd
import random
import re
random.seed(1)
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from textblob import Word

import string
from time import time

from bs4 import BeautifulSoup  
import re
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from nltk import sent_tokenize, word_tokenize, pos_tag


import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\10252946\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\10252946\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\10252946\AppData\Roaming\nltk_data...


C:\Users\10252946\Documents\Education\GATech\BigDataForHealth\Projects\venv\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\10252946\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
cur_dir = os.getcwd()
print("Current directory: {}".format(cur_dir))
# list files in the current directory
#print("Files in the current directory: {}".format(os.listdir(cur_dir)))
data_path = cur_dir+"/drugsCom"
train_path = data_path + "/drugsComTrain_raw.tsv"
test_path = data_path + "/drugsComTest_raw.tsv"
#rawdata_path = cur_dir+"/drugsCom"
#metadata_path = cur_dir+"/meta_Health_and_Personal_Care.json.gz"
print("Path of the raw data: {}".format(data_path))
print("Files in the drugCom directory: {}".format(os.listdir(data_path)))

Current directory: C:\Users\10252946\Documents\Education\GATech\BigDataForHealth\Projects
Path of the raw data: C:\Users\10252946\Documents\Education\GATech\BigDataForHealth\Projects/drugsCom
Files in the drugCom directory: ['drugsComTest_raw.tsv', 'drugsComTrain_raw.tsv']


In [3]:
train = pd.read_table(train_path, sep='\t')
test = pd.read_table(test_path, sep='\t')

In [4]:
train.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [5]:
train.shape

(161297, 7)

In [6]:
train['drugName'].nunique()

3436

In [7]:
documents = train['review'].values

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
documents_train, documents_test = train_test_split(documents, test_size = 0.2, random_state = 42)

In [10]:
# clean the data set

verb_exp = ['VB', 'VBZ', 'VBP', 'VBD','VBN','VBG']
#porter_stemmer = PorterStemmer()
def clean_text(row):
    soup = BeautifulSoup(row, 'html.parser')
    #remove code
    for tag in soup.find_all('code'):
        tag.replaceWith(' ')
        
    raw = soup.get_text()
    #remove link
    raw_no_link = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', raw)
    #remove email
    no_link_email = re.sub(r'[\w\.-]+@[\w\.-]+[\.][com|org|ch|uk]{2,3}', "", raw_no_link)
    #remove whitespace
    tab_text = '\t\n\r\x0b\x0c'
    no_link_email_space = "".join([ch for ch in no_link_email if ch not in set(tab_text)])
    #remove fomula
    reg = '(\$.+?\$)|((\\\\begin\{.+?\})(.+?)(\\\\end\{(.+?)\}))'
    raw = re.sub(reg, "", no_link_email_space, flags=re.IGNORECASE)   
    return(raw)

def lem_stop(row):
    row = row.lower()
    #remove numbers
    raw = re.sub('[0-9]+?', ' ', row) 
    # remove punctuation
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    raw = regex.sub(' ', raw)
    #clean out the characters left out after the above step, like we’re, I’m, It’s, i.e.
    raw = re.sub('( s )|( re )|( m )|( i e )',' ',raw) 
    # lementize
    row_t = TextBlob(raw)
    raw = []
    for word, pos in row_t.tags:
        if pos in verb_exp:
            word = Word(word)
            word = word.lemmatize("v")
        else:
            word = Word(word)
            word = word.lemmatize()
        raw.append(word)
    clean = ' '.join(raw)      
    stop_words = set(stopwords.words('english'))
    # remove stop words
    cleaned_text = " ".join([word for word in word_tokenize(clean) if word not in stop_words])      
    return(cleaned_text)

In [11]:
# get the clean review
review_rm = train['review'].map(lambda i: clean_text(i))

In [12]:
review_clean = review_rm.map(lambda i: lem_stop(i))

In [13]:
def sentiment_ana(i):
    i_tr = TextBlob(i)
    return(i_tr.sentiment[0], i_tr.sentiment[1])

In [14]:

sentiment = review_clean.map(lambda i: sentiment_ana(i))


In [53]:
print(sentiment)

0                                          (0.0, 0.0)
1                    (0.18802083333333336, 0.4828125)
2          (0.13848484848484852, 0.32747474747474753)
3           (0.35714285714285715, 0.6071428571428571)
4           (0.17233333333333328, 0.3896666666666667)
5          (0.10208333333333333, 0.28541666666666665)
6           (0.1111111111111111, 0.16666666666666666)
7                      (0.059166666666666666, 0.2825)
8                           (0.2, 0.5166666666666666)
9                      (0.04375000000000002, 0.34375)
10           (0.21000000000000002, 0.653888888888889)
11                       (0.2525, 0.4583333333333333)
12          (-0.17777777777777773, 0.611111111111111)
13        (-0.036574074074074064, 0.5416666666666665)
14         (0.05286796536796535, 0.49983766233766236)
15          (0.19722222222222227, 0.5981481481481482)
16        (-0.038636363636363635, 0.4212121212121212)
17         (0.30833333333333335, 0.47222222222222215)
18           (0.179166666666

In [16]:
senti_pol, senti_sub = [list(t) for t in zip(*sentiment)]

In [29]:
print(review_clean.shape)

(161297,)


In [17]:
train['Sentimental_Polarity'] = senti_pol
train['Sentimental_Subjectivity'] = senti_sub

In [18]:
train.shape

(161297, 9)

In [22]:
train.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,Sentimental_Polarity,Sentimental_Subjectivity
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,0.000000,0.000000
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,0.188021,0.482812
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,0.138485,0.327475
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,0.357143,0.607143
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,0.172333,0.389667


In [24]:
train.to_csv("drugsTrainingDataWithFeatures.csv", sep='\t', encoding='utf-8')

Run sentimental analysis on Test Data i.e. drugsComTest

In [25]:
# get the clean review
review_rm_test = test['review'].map(lambda i: clean_text(i))

In [26]:
review_clean_test = review_rm_test.map(lambda i: lem_stop(i))

In [29]:
def sentiment_ana(i):
    i_tr = TextBlob(i)
    return(i_tr.sentiment[0], i_tr.sentiment[1])

In [30]:
sentiment_test = review_clean_test.map(lambda i: sentiment_ana(i))

In [31]:
senti_pol_test, senti_sub_test = [list(t) for t in zip(*sentiment_test)]

In [32]:
test['Sentimental_Polarity'] = senti_pol_test
test['Sentimental_Subjectivity'] = senti_sub_test

In [33]:
test.shape

(53766, 9)

In [34]:
test.to_csv("drugsTestingDataWithFeatures.csv", sep='\t', encoding='utf-8')